In [ ]:
!pip install -q duckdb dbt-duckdb ducklake-delta-exporter  --upgrade
import sys
sys.exit(0)

In [1]:
import os
os.environ['DBT_SCHEMA']              = 'aemo'  
os.environ['ROOT_PATH']               = 'abfss://duckrun@onelake.dfs.fabric.microsoft.com/dbt.Lakehouse'
os.environ['download_limit']          = '10'

In [ ]:
!git clone https://github.com/djouallah/dbt.git /tmp/dbt

In [ ]:
import duckdb

root_path = os.environ['ROOT_PATH']
remote_blob = f"{root_path}/Files/metadata/data_0.db"
local_path = "/tmp/ducklake_metadata.db"

conn = duckdb.connect()

blob_exists = conn.execute(f"SELECT count(*) FROM glob('{remote_blob}')").fetchone()[0] > 0

if blob_exists:
    conn.execute(f"COPY (SELECT content FROM read_blob('{remote_blob}')) TO '{local_path}' (FORMAT BLOB)")
    size = conn.execute(f"SELECT octet_length(content) FROM read_blob('{local_path}')").fetchone()[0]
    print(f"Metadata downloaded: {size:,} bytes")
else:
    print("No remote metadata blob found — first run, starting fresh")

conn.close()

In [ ]:
!cd /tmp/dbt && dbt run && dbt test

In [2]:
from ducklake_delta_exporter import generate_latest_delta_log
generate_latest_delta_log('/tmp/ducklake_metadata.db')

  needs_export: 7 tables

📦 Exporting 7 tables...

  Processing aemo.fct_summary...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

  ✅ aemo.fct_summary: exported snapshot 17 (2 files)

  Processing aemo.fct_scada_today...


...

  ✅ aemo.dim_duid: exported snapshot 5 (1 files)

  Processing aemo.dim_calendar...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

  ✅ aemo.dim_calendar: exported snapshot 3 (1 files)

🎉 Export completed!


In [3]:
import duckdb
duckdb.sql(f"""select max(cutoff), count(*) from delta_scan
('{os.environ['ROOT_PATH']}/Tables/{os.environ['DBT_SCHEMA']}/fct_summary') 
 """).show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────────┬──────────────┐
│       max(cutoff)        │ count_star() │
│ timestamp with time zone │    int64     │
├──────────────────────────┼──────────────┤
│ 2026-02-09 00:00:00+10   │      1319522 │
└──────────────────────────┴──────────────┘

